In [1]:
import win32com.client
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

1


In [2]:
import win32com.client
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
print(instCpStockCode.GetData(1, 0))

동화약품


In [7]:
# import win32com.client

# # Create object
# instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

# # SetInputValue
# instStockChart.SetInputValue(0, "A012450") #한화에어로스페이스
# instStockChart.SetInputValue(1, ord('1'))
# instStockChart.SetInputValue(2, 20230626)
# instStockChart.SetInputValue(3, 20210627)
# instStockChart.SetInputValue(4, 10)
# instStockChart.SetInputValue(5, (0, 1, 2, 3, 4, 5, 6,7,8))
# instStockChart.SetInputValue(6, ord('m')) #분 당
# instStockChart.SetInputValue(9, ord('1')) #1 분

# # BlockRequest
# instStockChart.BlockRequest()

# # GetHeaderValue
# numData = instStockChart.GetHeaderValue(3)
# numField = instStockChart.GetHeaderValue(1)

# # GetDataValue
# for i in range(numData):
#     for j in range(numField):
#         print(instStockChart.GetDataValue(j, i), end=" ")
#     print("")

20230626 1530 131600 131600 131600 131600 0 17009 
20230626 1520 131400 131600 131400 131500 0 2242 
20230626 1519 131400 131500 131400 131500 0 1411 
20230626 1518 131400 131500 131400 131500 0 991 
20230626 1517 131300 131500 131300 131400 0 1844 
20230626 1516 131300 131400 131300 131300 0 341 
20230626 1515 131400 131500 131300 131300 0 2038 
20230626 1514 131100 131400 131000 131400 0 2709 
20230626 1513 131100 131100 131000 131000 0 868 
20230626 1512 131100 131200 131000 131100 0 1154 
20230626 1511 131500 131500 131100 131100 0 6594 
20230626 1510 131500 131500 131400 131500 0 947 
20230626 1509 131600 131600 131400 131500 0 1918 
20230626 1508 131500 131600 131500 131500 0 1290 
20230626 1507 131500 131600 131500 131600 0 1094 
20230626 1506 131500 131700 131500 131500 0 2875 
20230626 1505 131500 131600 131400 131600 0 1478 
20230626 1504 131500 131600 131400 131600 0 1439 
20230626 1503 131600 131700 131500 131500 0 1596 
20230626 1502 131400 131600 131300 131500 0 2444 
202

In [9]:
#종목정보 가져오기(최초)

import win32com.client
import pandas as pd 

rows = list()

CPE_MARKET_KIND = {'KOSPI':1,  'KOSDAQ':2}
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")

for key, value in CPE_MARKET_KIND.items():
    codeList = instCpCodeMgr.GetStockListByMarket(value)
    for code in codeList:
        name = instCpCodeMgr.CodeToName(code)
        sectionKind = instCpCodeMgr.GetStockSectionKind(code)
        row = [code, name, key, sectionKind]
        rows.append(row)

print('모든 종목을 불러왔습니다')

stockitems = pd.DataFrame(data= rows, columns=['code','name', 'section','sectionKind'])
stockitems.loc[stockitems['sectionKind'] == 10, 'section'] =   'ETF'
stockitems.to_csv('stockitems.csv', index=False)

print('파일을 저장하였습니다.')

모든 종목을 불러왔습니다
파일을 저장하였습니다.


In [10]:
##주가정보 가져오기  (최초 생성)

import win32com.client
import pandas as pd 
import time
import datetime

column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close',
                     'vol', 'value', 'n_stock', 'agg_price', 'foreign_rate','agency_buy', 'agency_netbuy']

stockitems = pd.read_csv('stockitems.csv')

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

row = list(range(len(column_dailychart)))
rows = list()
instStockChart.SetInputValue(0, "A012450")
instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(3, '20220627')
instStockChart.SetInputValue(5, (0, 1, 2, 3, 4, 5, 8, 9, 12, 13, 17, 20, 21))
instStockChart.SetInputValue(6, ord('m'))
instStockChart.SetInputValue(9, ord('1'))
    
for idx, stockitem in stockitems.iterrows():

    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
    instStockChart.SetInputValue(0, stockitem['code'])

    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)

    # GetDataValue
    for i in range(numData):
        row[0] =  stockitem['code']
        row[1] = stockitem['section']  # 코스피, 코스닥, ETF 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시가
        row[4] = instStockChart.GetDataValue(2, i)  # 고가
        row[5] = instStockChart.GetDataValue(3, i)  # 저가
        row[6] = instStockChart.GetDataValue(4, i)  # 종가
        row[7] = instStockChart.GetDataValue(5, i)  # 거래량
        row[8] = instStockChart.GetDataValue(6, i)  # 거래대금
        row[9] = instStockChart.GetDataValue(7, i)  # 상장주식수
        row[10] = instStockChart.GetDataValue(8, i)  # 시가총액
        row[11] = instStockChart.GetDataValue(9, i)  # 외국인 보율비율
        row[12] = instStockChart.GetDataValue(10, i)  # 기관순매수
        row[13] = instStockChart.GetDataValue(11, i)  # 기관누적순매수
        rows.append(list(row))  
        
print('데이터를 모두 불러왔습니다.')

dailychart= pd.DataFrame(data = rows, columns= column_dailychart)
dailychart[['value' ,'agg_price']] = (dailychart[['value' ,'agg_price']]/1000000).astype(int)
dailychart = dailychart.sort_values(by=['code','date'])
dailychart.to_csv('dailychart.csv', index=False)

print('모든 데이터를 저장하였습니다.')

A000020 동화약품 남은 요청 :  60
A000040 KR모터스 남은 요청 :  59
A000050 경방 남은 요청 :  58
A000070 삼양홀딩스 남은 요청 :  57
A000075 삼양홀딩스우 남은 요청 :  56
A000080 하이트진로 남은 요청 :  55
A000087 하이트진로2우B 남은 요청 :  54
A000100 유한양행 남은 요청 :  53
A000105 유한양행우 남은 요청 :  52
A000120 CJ대한통운 남은 요청 :  51
A000140 하이트진로홀딩스 남은 요청 :  50
A000145 하이트진로홀딩스우 남은 요청 :  49
A000150 두산 남은 요청 :  48
A000155 두산우 남은 요청 :  47
A000157 두산2우B 남은 요청 :  46
A000180 성창기업지주 남은 요청 :  45
A000210 DL 남은 요청 :  44
A000215 DL우 남은 요청 :  43
A000220 유유제약 남은 요청 :  42
A000225 유유제약1우 남은 요청 :  41
A000227 유유제약2우B 남은 요청 :  40
A000230 일동홀딩스 남은 요청 :  39
A000240 한국앤컴퍼니 남은 요청 :  38
A000270 기아 남은 요청 :  37
A000300 대유플러스 남은 요청 :  36
A000320 노루홀딩스 남은 요청 :  35
A000325 노루홀딩스우 남은 요청 :  34
A000370 한화손해보험 남은 요청 :  33
A000390 삼화페인트 남은 요청 :  32
A000400 롯데손해보험 남은 요청 :  31
A000430 대원강업 남은 요청 :  30
A000480 조선내화 남은 요청 :  29
A000490 대동 남은 요청 :  28
A000500 가온전선 남은 요청 :  27
A000520 삼일제약 남은 요청 :  26
A000540 흥국화재 남은 요청 :  25
A000545 흥국화재우 남은 요청 :  24
A000547 흥국화재2우B 남은 요청 :  23
A000590 CS홀딩스 남은 

In [ ]:
obj.SetInputValue 0, "A005930" 
obj.SetInputValue 1, Asc("1") #기간 선택
obj.SetInputValue 2, 20200320 '종료일자
obj.SetInputValue 3, 20140101 '시작일자
obj.SetInputValue 5, Items
obj.SetInputValue 6, Asc("m") #분봉
obj.SetInputValue 7, 1 #1분